# Tosho Recommender (Attention Visualization)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pickle
import gensim
import janome
from janome.tokenizer import Tokenizer as ja_tokenizer
import re
from IPython.display import HTML
from IPython import display as disp
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
from model import *

## Load Model

In [3]:
def load_obj(filename):
    with open(filename, 'rb') as handler:
        return pickle.load(handler)

# Load tokenizer
tokenizer = load_obj('model/tosho_recommender_word_tokenizer.pkl')
word_index = load_obj('model/tosho_recommender_word_index.pkl')
itos = {v: k for k, v in word_index.items()}

# Load Book2Vec (book-level representation)
b2v = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format('model/book2vec', binary=False)

# Load book descriptions and titles
text_pd = pd.read_csv('data/tosho_processed_clean.csv.bz2', sep='\t', compression='bz2')

# Load tf.keras model
model = gru_model(
                     embedding_dim=300,
                     dropout_rate=0.209,
                     rnn_unit=194,
                     input_shape=(500,),
                     num_features=20000+1,
                     share_gru_weights_on_book=True,
                     use_attention_on_book=True,
                     use_attention_on_user=True,
                     use_batch_norm=False,
                     is_embedding_trainable=False,
                     final_activation='tanh',
                     final_dimension=392,
                     embedding_matrix=np.zeros((20001, 300)))

model.compile(loss='cosine_similarity',
                  optimizer=Adam(lr=0.0044))

x = np.ones((1, 500))
y = np.ones((1, 392))

model.train_on_batch([x, x, x, x], y)

# Load the state of the old model
model.load_weights('model/tosho_recommender_7')

/opt/conda/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
def clean_text(text:str):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s{1,}', '', text)
    
    text = re.sub(r'内容紹介', '', text)
    text = re.sub(r'出版社からのコメント', '', text)
    text = re.sub(r'商品の説明をすべて表示する', '', text)
    text = re.sub(r'内容（「MARC」データベースより）', '', text)
    text = re.sub(r'内容（「BOOK」データベースより）', '', text)

    non_japanese = re.compile(r"[^0-9\-ぁ-ヶ亜-黑ー]")
    text = re.sub(non_japanese, ' ', text)

    return text.strip()

In [5]:
j_tokenizer = ja_tokenizer()

def wakati_reading(text:str):
    tokens = j_tokenizer.tokenize(text.replace("'", "").lower())
    
    exclude_pos = [u'助動詞']
    
    #分かち書き
    tokens_w_space = ""
    for token in tokens:
        partOfSpeech = token.part_of_speech.split(',')[0]
        
        if partOfSpeech not in exclude_pos:
            tokens_w_space = tokens_w_space + " " + token.surface

    tokens_w_space = tokens_w_space.strip()
    tokens_w_space = re.sub(r'\s{2,}', ' ', tokens_w_space)
    
    return tokens_w_space

In [6]:
def preprocess_text(text:str):
    MAX_SEQUENCE_LENGTH = 500
    
    text = clean_text(text)
    text = wakati_reading(text)

    x = tokenizer.texts_to_sequences([text])
    x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)
    
    return x

## Attention Model

### Attention weights for token-level representation

In [7]:
token_layer_outputs = [layer.output for layer in model.layers[:-2]]

In [8]:
token_layer_outputs

[<tf.Tensor 'input_1:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'input_3:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'word_embed_1/Identity:0' shape=(None, None, 300) dtype=float32>,
 <tf.Tensor 'word_embed_2/Identity:0' shape=(None, None, 300) dtype=float32>,
 <tf.Tensor 'word_embed_3/Identity:0' shape=(None, None, 300) dtype=float32>,
 <tf.Tensor 'word_embed_4/Identity:0' shape=(None, None, 300) dtype=float32>,
 [<tf.Tensor 'gru_with__attn/Identity:0' shape=(None, None, 1) dtype=float32>,
  <tf.Tensor 'gru_with__attn/Identity_1:0' shape=(None, 388) dtype=float32>]]

In [9]:
token_attn_model = models.Model(inputs=model.input, outputs=token_layer_outputs[-1])

### Attention weights for book-level representation

In [10]:
book_layer_outputs = [layer.output for layer in model.layers[:-1]]

In [11]:
book_layer_outputs

[<tf.Tensor 'input_1:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'input_3:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(None, None) dtype=float32>,
 <tf.Tensor 'word_embed_1/Identity:0' shape=(None, None, 300) dtype=float32>,
 <tf.Tensor 'word_embed_2/Identity:0' shape=(None, None, 300) dtype=float32>,
 <tf.Tensor 'word_embed_3/Identity:0' shape=(None, None, 300) dtype=float32>,
 <tf.Tensor 'word_embed_4/Identity:0' shape=(None, None, 300) dtype=float32>,
 [<tf.Tensor 'gru_with__attn/Identity:0' shape=(None, None, 1) dtype=float32>,
  <tf.Tensor 'gru_with__attn/Identity_1:0' shape=(None, 388) dtype=float32>],
 [<tf.Tensor 'gru_with__attn_1_1/Identity:0' shape=(None, 4, 1) dtype=float32>,
  <tf.Tensor 'gru_with__attn_1_1/Identity_1:0' shape=(None, 388) dtype=float32>]]

In [12]:
book_attn_model = models.Model(inputs=model.input, outputs=book_layer_outputs[-1])

## Manually Input 4 Books to Predict

In [13]:
# 名探偵コナン (1)
book1 = """
▼第1話/平成のホームズ
▼第2話/小さくなった名探偵
▼第3話/仲間はずれの名探偵
▼第4話/6本目の煙突
▼第5話/もう一人の犯人
▼第6話/迷探偵を名探偵に
▼第7話/血ぬられたアイドル
▼第8話/あなたに似た人
▼第9話/不幸な誤解
"""

# おしりたんてい かいとうと ねらわれた はなよめ (おしりたんていファイル 8)
book2 = """
はたして 今回の かいとうUの ねらいは…!?

おしりたんていの事務所に、とつぜんあらわれた
なぞのいらいにん。だが、そのしょうたいをたちどころに
見抜いたわれらがおしりたんてい。なぞの依頼人が
持ち込んだのは、なんとかいとうUからの予告状だった!

いにしえからの、けっこんの儀式にかくされたお宝を
かいとうUからまもるため、今回もおしりたんていの
名推理が冴えわたる。

迷路や、絵探しなど、
おしりたんていといっしょに謎を解きながら、
真実にせまる、本格的推理読み物シリーズです。

今回も、「かいとうと ねらわれた はなよめ」
「おりの なかの けいかく」の2話収録。

何度読んでも発見がある
推理小説の入り口にも最適な
知的好奇心をくすぐる1冊です。
"""

# がっこうのおばけずかん (どうわがいっぱい)
book3 = """
怖いけどおもしろい、「図鑑」という名の童話、「おばけずかん」シリーズ最新刊。「ひょうほんがいこつ」「おんがくしつのベートーベン」「トイレのはなこさん」などなど、毎日通っている学校にもこわ~いおばけはいっぱいいるけど、このお話を読めば、だいじょうぶ!


昔から知られているおばけがいっぱい登場する、「図鑑」という名前の童話「おばけずかん」シリーズの新刊です。
それぞれのおばけが、どんなふうに怖いのか。そうならないためには、どうしたらだいじょうぶなのかを、ユーモラスな短いお話仕立てで紹介しています。
登場するおばけはちょっと怖いけど、ちゃんと対応してあげると、意外になさけなくて、かわいいところもあったりします。
怖くて、笑えて、最後はホッとできる。「こわいけど、おもしろい」、新しいおばけの童話シリーズ第4弾です。
読者に身近な小学校を舞台に、『トイレのはなこさん』のような新しいおばけや、オリジナルのおばけ『れんぞくこうちょうせんせい』も登場。シリーズの新しい展開を見せる一冊です。

●この本に登場するおばけ

ひょうほんがいこつ
おんがくしつの ベートーベン
トイレの はなこさん
こうていの にのみやきんじろう
ゆうれいアナウンサー
れんぞくこうちょうせんせい
みつめの 六ねんせい
まよなかの まぼろしうんどうかい

※漢字は使用しません
"""

# 星のカービィ 虹の島々を救え!の巻
book4 = """
カービィは、リック&カイン&クーと虹の島々を救う大冒険に出発!

カービィの友だち、リック&カイン&クーがやってきた!
三人が住む虹の島々に雨が降らなくなり、困っているらしい。
カービィはメタナイトやデデデ大王と虹の島々へ!!
原因を知る女の子・ピリカと出会い
雲の上に向かうと、そこにいたのは
カービィとも仲良しな友だちの、グーイだった。
なぜか、グーイはカービィたちに襲いかかってきて!?
いったい、何が起こったのか……?
事件の解決に、カービィがいどむ!!
"""
book1 = preprocess_text(book1)
book2 = preprocess_text(book2)
book3 = preprocess_text(book3)
book4 = preprocess_text(book4)

book_list = [book1, book2, book3, book4]

In [14]:
token_attn_preds_1 = token_attn_model.predict([book1, book2, book3, book4])[0]

In [15]:
token_attn_preds_2 = token_attn_model.predict([book2, book1, book3, book4])[0]

In [16]:
token_attn_preds_3 = token_attn_model.predict([book3, book2, book1, book4])[0]

In [17]:
token_attn_preds_4 = token_attn_model.predict([book4, book2, book3, book1])[0]

In [18]:
book_attn_preds = book_attn_model.predict(book_list)[0]

### Sanity check

In [20]:
np.sum(token_attn_preds_1[0][:,0]), np.sum(token_attn_preds_2[0][:,0]), np.sum(token_attn_preds_3[0][:,0]), np.sum(token_attn_preds_4[0][:,0])

(1.0000002, 1.0000001, 1.0000001, 1.0000002)

In [21]:
np.sum(book_attn_preds[0][:,0])

0.99999994

## Attention Visualization

In [22]:
def highlight(word, attn, scale):
    html_color = '#%02X%02X%02X' % (255, max(0, int(255*(1 - attn*scale))), max(0, int(255*(1 - attn*scale))))
    return '<span style="background-color: {}">{}</span>'.format(html_color, word)

def mk_html(sentence, attns):
    html = ""
    scale = attns.shape[0] * 0.1
    for word, attn in zip(sentence, attns):
        html += ' ' + highlight(
            word,
            attn,
            scale
        )
    return html + "<br><br>\n"

### Visualize attention weights on each token

In [27]:
def token_attention_vis(token_model, book_vec):
    non_zeros = book_vec[book_vec > 0].shape[0]
    text_list = [itos[i] for i in book_vec[0] if i != 0]

    # zero padding might have very small attn weights, so rescale
    attn_for_words = token_model[0][:,0][500-non_zeros:]
    scale_to_1 = 1/np.sum(attn_for_words)

    attn_vis = mk_html(text_list, attn_for_words*scale_to_1)
    disp.display(HTML(attn_vis))

In [28]:
token_attention_vis(token_attn_preds_1, book1)

In [29]:
token_attention_vis(token_attn_preds_2, book2)

In [30]:
token_attention_vis(token_attn_preds_3, book3)

In [31]:
token_attention_vis(token_attn_preds_4, book4)

### Determine which book had the highest attention weight

In [40]:
text_list = ['1st book', '2nd book', '3rd book', '4th book']
attn_vis = mk_html(text_list, book_attn_preds[0][:,0])
disp.display(HTML(attn_vis))